In [1]:
from keras import layers
from keras import models

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = models.Sequential()

### Load Data

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('data/uci-news-aggregator.csv')

In [5]:
data[['CATEGORY', 'TITLE']].head()

,CATEGORY,TITLE
0,b,"Fed official says weak data caused by weather,..."
1,b,Fed's Charles Plosser sees high bar for change...
2,b,US open: Stocks fall after Fed official hints ...
3,b,"Fed risks falling 'behind the curve', Charles ..."
4,b,Fed's Plosser: Nasty Weather Has Curbed Job Gr...


### First create a baseline

In [6]:
data_by_category = data.groupby('CATEGORY')

In [7]:
category_count = data_by_category['ID'].agg(['count'])

In [8]:
sorted_count = category_count.sort_values('count', ascending=False)

In [9]:
# take the highest occuring item and suppose that we predicted all records to be this category
max_value = sorted_count.max()
total_count = len(data)

In [10]:
base_line_value = max_value / total_count

In [11]:
base_line_accuracy = base_line_value[0]
print('Baseline accuracy to beat: {:f}%'.format((base_line_accuracy * 100)))

Baseline accuracy to beat: 36.094257%


### Define targets and data

In [12]:
from keras.preprocessing.text import Tokenizer

In [13]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['TITLE'].values)
x = tokenizer.texts_to_matrix(data['TITLE'].values, mode='binary')

In [14]:
from keras.preprocessing.text import one_hot
from keras.utils.np_utils import to_categorical

In [15]:
tokenizer = Tokenizer(num_words=4)
tokenizer.fit_on_texts(data['CATEGORY'].values)
y = tokenizer.texts_to_matrix(data['CATEGORY'].values, mode='binary').astype('int8')

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=0.2)

### Create simple dense neural network

In [28]:
model = models.Sequential()

In [29]:
model.add(layers.Dense(32, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(len(category_count), activation='softmax'))

In [30]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [31]:
model.fit(
    x_train,
    y_train,
    epochs = 3,
    batch_size=512,
    validation_data=(x_validation, y_validation)
)

Train on 337935 samples, validate on 84484 samples
Epoch 1/3
337935/337935 [==============================] - 71s 210us/step - loss: 0.2689 - acc: 0.8188 - val_loss: 0.1520 - val_acc: 0.8379
Epoch 2/3
337935/337935 [==============================] - 68s 201us/step - loss: 0.1370 - acc: 0.8428 - val_loss: 0.1425 - val_acc: 0.8407
Epoch 3/3
337935/337935 [==============================] - 70s 208us/step - loss: 0.1274 - acc: 0.8462 - val_loss: 0.1384 - val_acc: 0.8422
